# ✂️ Tokenization: Hoe AI tekst leest

Taalmodellen zoals ChatGPT, Claude en Gemini lezen geen woorden — ze lezen **tokens**.

Een token is een stukje tekst: soms een heel woord, soms een deel van een woord, soms een leesteken.

In dit notebook ontdek je:
1. Hoe tekst wordt opgesplitst in tokens
2. Waarom sommige woorden in stukjes worden geknipt
3. Hoe padding en truncation werken

---
**Instructie:** Voer iedere cel uit met **Shift+Enter**

⚠️ De eerste cel kan even duren (~30 seconden) omdat er een tokenizer wordt gedownload.

In [1]:
# Installeer de benodigde library (duurt even bij eerste keer)
!pip install transformers -q

## Stap 1: Tekst tokenizen

We laden de **BERT tokenizer** — dezelfde tokenizer die gebruikt wordt in veel AI-modellen.

In [2]:
from transformers import BertTokenizer

# Laad de BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Probeer een zin
tekst = "PyTorch and Hugging Face make deep learning simple."

# Tokenize!
tokens = tokenizer.tokenize(tekst)

print(f"Originele tekst:  {tekst}")
print(f"Aantal woorden:   {len(tekst.split())}")
print(f"\nTokens:           {tokens}")
print(f"Aantal tokens:    {len(tokens)}")

c:\Users\sybra\OneDrive\Bureaublad\neural_nets\neural_networks_training_advanced_ai\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\sybra\OneDrive\Bureaublad\neural_nets\neural_networks_training_advanced_ai\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sybra\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For mo

Originele tekst:  PyTorch and Hugging Face make deep learning simple.
Aantal woorden:   8

Tokens:           ['p', '##yt', '##or', '##ch', 'and', 'hugging', 'face', 'make', 'deep', 'learning', 'simple', '.']
Aantal tokens:    12


### Wat valt je op?

- `"PyTorch"` is opgesplitst in `['p', '##yt', '##or', '##ch']` — het model kent het woord niet als geheel
- `"##"` betekent: dit stukje hoort bij het vorige token
- Bekende woorden zoals `"and"`, `"make"`, `"deep"` blijven heel
- De punt `.` is een apart token

## Stap 2: Van tokens naar getallen

Het neural network werkt niet met tekst maar met **getallen**. Elk token heeft een uniek ID-nummer.

In [3]:
# Zet tokens om naar ID-nummers
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print("Token → ID-nummer:")
print("-" * 35)
for token, id in zip(tokens, token_ids):
    print(f"  {token:15s} → {id}")

print(f"\nBERT heeft een vocabulaire van {tokenizer.vocab_size:,} tokens!")

Token → ID-nummer:
-----------------------------------
  p               → 1052
  ##yt            → 22123
  ##or            → 2953
  ##ch            → 2818
  and             → 1998
  hugging         → 17662
  face            → 2227
  make            → 2191
  deep            → 2784
  learning        → 4083
  simple          → 3722
  .               → 1012

BERT heeft een vocabulaire van 30,522 tokens!


## Stap 3: Speciale tokens

BERT voegt automatisch speciale tokens toe:
- `[CLS]` — markering van het begin van de tekst
- `[SEP]` — markering van het einde van de tekst
- `[PAD]` — opvulling om alle zinnen even lang te maken

In [4]:
# Volledige encoding met speciale tokens en padding
encoded = tokenizer(
    tekst,
    padding='max_length',  # Vul aan tot max_length
    truncation=True,       # Knip af als het te lang is
    max_length=20          # Maximale lengte
)

# Laat zien wat eruit komt
alle_tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'])

print("Volledige token-reeks (met padding tot lengte 20):")
print("-" * 50)
for i, (token, id, mask) in enumerate(zip(
    alle_tokens, encoded['input_ids'], encoded['attention_mask']
)):
    status = '✅' if mask == 1 else '⬜ (padding)'
    print(f"  [{i:2d}] {token:15s}  ID: {id:5d}  {status}")

print(f"\nAttention mask: {encoded['attention_mask']}")
print("(1 = echte tekst, 0 = opvulling die het model negeert)")

Volledige token-reeks (met padding tot lengte 20):
--------------------------------------------------
  [ 0] [CLS]            ID:   101  ✅
  [ 1] p                ID:  1052  ✅
  [ 2] ##yt             ID: 22123  ✅
  [ 3] ##or             ID:  2953  ✅
  [ 4] ##ch             ID:  2818  ✅
  [ 5] and              ID:  1998  ✅
  [ 6] hugging          ID: 17662  ✅
  [ 7] face             ID:  2227  ✅
  [ 8] make             ID:  2191  ✅
  [ 9] deep             ID:  2784  ✅
  [10] learning         ID:  4083  ✅
  [11] simple           ID:  3722  ✅
  [12] .                ID:  1012  ✅
  [13] [SEP]            ID:   102  ✅
  [14] [PAD]            ID:     0  ⬜ (padding)
  [15] [PAD]            ID:     0  ⬜ (padding)
  [16] [PAD]            ID:     0  ⬜ (padding)
  [17] [PAD]            ID:     0  ⬜ (padding)
  [18] [PAD]            ID:     0  ⬜ (padding)
  [19] [PAD]            ID:     0  ⬜ (padding)

Attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
(1 = echte tekst, 0 

## Stap 4: Terug van getallen naar tekst

We kunnen de getallen ook weer terugvertalen naar tekst.

In [5]:
# Decodeer terug naar leesbare tekst
decoded = tokenizer.decode(encoded['input_ids'], skip_special_tokens=True)

print(f"Origineel:  {tekst}")
print(f"Gedecodeerd: {decoded}")
print(f"\n(Merk op: hoofdletters zijn verdwenen — BERT werkt met 'uncased' tekst)")

Origineel:  PyTorch and Hugging Face make deep learning simple.
Gedecodeerd: pytorch and hugging face make deep learning simple.

(Merk op: hoofdletters zijn verdwenen — BERT werkt met 'uncased' tekst)


## 🧪 Experimenteer zelf!

Probeer verschillende zinnen te tokenizen. Wat gebeurt er met:
- Nederlandse woorden?
- Spelfouten?
- Getallen?
- Emoji's?

In [6]:
# Probeer je eigen tekst!
mijn_tekst = "Dit is een Nederlandse zin over kunstmatige intelligentie."  # ← Pas dit aan

mijn_tokens = tokenizer.tokenize(mijn_tekst)
print(f"Tekst:  {mijn_tekst}")
print(f"Tokens: {mijn_tokens}")
print(f"Aantal tokens: {len(mijn_tokens)}")

Tekst:  Dit is een Nederlandse zin over kunstmatige intelligentie.
Tokens: ['di', '##t', 'is', 'ee', '##n', 'ned', '##erland', '##se', 'z', '##in', 'over', 'kunst', '##mat', '##ige', 'intelligent', '##ie', '.']
Aantal tokens: 17


In [7]:
# Vergelijk: correct gespeld vs. spelfout
woorden = ["intelligence", "inteligence", "inttellligennce", "Amsterdam", "Amstrdm"]

print("Hoe gaat de tokenizer om met spelfouten?")
print("-" * 50)
for woord in woorden:
    toks = tokenizer.tokenize(woord)
    print(f"  {woord:20s} → {toks}")

Hoe gaat de tokenizer om met spelfouten?
--------------------------------------------------
  intelligence         → ['intelligence']
  inteligence          → ['intel', '##igen', '##ce']
  inttellligennce      → ['int', '##tell', '##li', '##gen', '##nce']
  Amsterdam            → ['amsterdam']
  Amstrdm              → ['am', '##st', '##rd', '##m']
